In [ ]:
import $ivy.`org.flinkextended::flink-scala-api:1.17.1_1.1.0`

import org.apache.flinkx.api._
import org.apache.flinkx.api.serializers._

In [ ]:
import $ivy.`org.apache.flink:flink-table-api-java:1.17.1`
import $ivy.`org.apache.flink:flink-table-api-java-bridge:1.17.1`
import $ivy.`org.apache.flink:flink-table-planner-loader:1.17.1`
import $ivy.`org.apache.flink:flink-table-runtime:1.17.1`

import $ivy.`org.apache.flink:flink-clients:1.17.1`
import $ivy.`org.apache.flink:flink-shaded-hadoop-2-uber:2.8.3-10.0`
import $ivy.`org.apache.flink:flink-runtime-web:1.17.1`

import $ivy.`org.slf4j:slf4j-log4j12:1.7.15`

import $ivy.`org.apache.hadoop:hadoop-mapreduce-client-core:2.8.3`
import $ivy.`org.apache.hadoop:hadoop-mapreduce-client-common:2.8.3`
import $ivy.`org.apache.hadoop:hadoop-mapreduce-client-jobclient:2.8.3`
import $ivy.`org.apache.paimon:paimon-flink-1.17:0.5.0-incubating`

import org.apache.flink.table.api.bridge.java.StreamTableEnvironment
import org.apache.flink.table.api._
import org.apache.flink.configuration.Configuration
import org.apache.flink.configuration.ConfigConstants
import org.apache.flink.configuration.RestOptions.BIND_PORT

import scala.collection.JavaConverters._

val config = Configuration.fromMap(
  Map(
    ConfigConstants.LOCAL_START_WEBSERVER -> "true",
    BIND_PORT.key -> "8082",
    "execution.checkpointing.interval" -> "10 s"
  ).asJava
)

val env = StreamExecutionEnvironment.createLocalEnvironmentWithWebUI(config)
val tEnv = StreamTableEnvironment.create(env.getJavaEnv)
val settings = EnvironmentSettings.newInstance().inStreamingMode()
  .withConfiguration(env.getJavaEnv.getConfiguration.asInstanceOf[Configuration])
  .build()
val table = TableEnvironment.create(settings)

In [ ]:
def runFlinkSql(sql: String) = 
  sql.split(";").map(_.trim).filterNot(_.isEmpty).foreach { s => 
    println(s"Executing: $s")
    table.executeSql(s).print
  }    

In [ ]:
runFlinkSql("""
    CREATE CATALOG my_catalog WITH (
        'type'='paimon',
        'warehouse'='file:/tmp/paimon'
    );
    USE CATALOG my_catalog
    """
)

In [ ]:
runFlinkSql("""
  use catalog my_catalog;
  CREATE TEMPORARY TABLE word_table (
    word STRING
  ) WITH (
    'connector' = 'datagen',
    'fields.word.length' = '1'
  )"""
)

In [ ]:
val result = table.executeSql("""
    INSERT INTO word_count
    SELECT word, COUNT(*) FROM word_table GROUP BY word
""")

In [ ]:
//import scala.jdk.OptionConverters._
result.getJobClient().toScala.foreach(status => print(status.getJobStatus.get))

In [ ]:
result.getJobClient().toScala.foreach(_.cancel())